In [1]:
import os
import boto3

session = boto3.Session(
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY"),
    aws_secret_access_key=os.getenv("AWS_SECRET_KEY"),
)

client = session.client("bedrock-runtime", "us-east-1")

In [2]:
from langchain.chat_models import BedrockChat
from langchain.prompts import ChatPromptTemplate


chat = BedrockChat(
    client=client,
    model_id="anthropic.claude-v2",
    model_kwargs={
        "temperature": 0.1,
    },
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            "Translate this sentence from {lang_a} to {lang_b}: {sentence}",
        ),
    ]
)

chain = prompt | chat

chain.invoke(
    {
        "lang_a": "English",
        "lang_b": "Icelandic",
        "sentence": "I love amazon!",
    }
)

c:\Users\ybr24\miniconda3\envs\homework5\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `BedrockChat` was deprecated in LangChain 0.0.34 and will be removed in 0.3. An updated version of the class exists in the langchain-aws package and should be used instead. To use it run `pip install -U langchain-aws` and import as `from langchain_aws import ChatBedrock`.
  warn_deprecated(


AIMessage(content='Hér er þýðingin á setningunni "I love amazon!" yfir á íslensku:\n\nÉg elska amazon!', response_metadata={'model_id': 'anthropic.claude-v2', 'usage': {'prompt_tokens': 23, 'completion_tokens': 40, 'total_tokens': 63}}, id='run-07856c1f-bafa-4db5-961c-7a6bea0e3365-0')